# MNIST dataset 파이토치 코드 예시

## 개발 목표
숫자 손글씨 이미지 데이터를 입력받아 그에 해당하는 숫자로 분류시키는 딥러닝 모델 개발

## 데이터 확보 및 정제
MNIST 데이터셋 활용

In [ ]:
# 실제 데이터
import torchvision
import numpy as np
import matplotlib.pyplot as plt

mnist = torchvision.datasets.MNIST('dataset/', train=False, download=True)

image, label = mnist[0]
print()
print(type(image))
print(np.array(image).shape)

plt.title(f"label: {label}")
plt.imshow(image, cmap='gray')
plt.show()

In [ ]:
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

# ToTensor: (28, 28) 의 PIL 이미지 데이터를 (1, 28, 28) 의 torch.Tensor 로 변경
train_dataset = torchvision.datasets.MNIST('dataset/', train=False, download=True, transform=ToTensor())
test_dataset = torchvision.datasets.MNIST('dataset/', train=False, download=True, transform=ToTensor())

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1024, shuffle=True)

## 모델 설계 및 선택
이미지 패턴 탐지에 적합한 convolution layer 를 활용, 이후 fully connected layer 를 통해 0 ~ 9 까지 10개의 카테고리 중 하나를 선택하도록 설계

In [ ]:
import torch
import torch.nn as nn


class SampleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm2d(16),  # BatchNormalization: 학습을 도와주는 역할
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )
        self.dropout = nn.Dropout(.5)  # Dropout: 확률적으로 신호를 차단하여, 오버피팅 예방
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, softmax=False):
        if x.dim() == 3:
            x = x.view(1, x.size(0), x.size(1), x.size(2))
        x = self.conv_block(x)
        x = self.dropout(x)
        if softmax:
            return self.softmax(self.fc(x))
        return self.fc(x)


## 모델 학습 및 검증


In [ ]:
# 모델 하이퍼 파라미터 설정

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPU 사용 가능할 시 사용, 그 외엔 CPU 사용
model = SampleModel().to(device)  # 모델 선언
criterion = nn.CrossEntropyLoss()  # CrossEntropyLoss: 다중 클래스 분류용 Loss
learning_rate = 0.0001  # 학습률 선택
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  # Adam Optimizer 선택: model의 파라미터에 대한 그래디언트 추적
epochs = 30  # 학습 에폭 수


In [ ]:
# 학습 전 모델 성능 체크
import random


def performance_check(model: nn.Module, dataset):
    # 데이터셋에서 랜덤하게 10개 데이터 추출 후 모델 예측결과와 비교
    # score 의 경우 softmax 의 값을 확률처럼 사용하였음
    sample_images = []
    sample_labels = []
    for i in random.choices(range(len(dataset)), k=10):
        sample_image, sample_label = dataset[i]
        sample_images.append(sample_image)
        sample_labels.append(sample_label)

    model.eval()
    plt.figure(figsize=(10, 6))
    with torch.no_grad():
        for i in range(10):
            plt.subplot(2, 5, i + 1)
            test_image = sample_images[i].to(device)
            model_result = model(test_image, softmax=True)
            pred_label = torch.argmax(model_result, dim=1).item()
            pred_score = torch.max(model_result).item()
            plt.title(f"pred_label: {pred_label}\nreal_label: {sample_labels[i]}\nscore: {pred_score * 100: .2f}%")
            plt.imshow(test_image.squeeze(0).cpu().numpy(), cmap='gray')
            plt.axis('off')
        plt.show()


performance_check(model, test_dataset)

In [ ]:
# 학습 및 검증 시작
train_loss = []
train_accuracy = []
test_accuracy = []

max_acc = 0.0
best_model = model.state_dict().copy()  # 모델 기록

for epoch in range(epochs):
    model.train()  # 학습 모드: Dropout, BatchNorm 등이 학습 시 동작 방식으로 전환됨

    running_loss = 0.0  # epoch 에 대한 train loss sum
    correct = 0  # epoch 에 대한 train 적중 횟수
    total = 0  # 총 train 데이터 수

    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  # 그래디언트 초기화
        outputs = model(images)  # 모델 출력 (순전파)
        loss = criterion(outputs, labels)  # 손실 측정
        loss.backward()  # Autograd를 사용해 손실(loss)에 대한 각 파라미터의 gradient 계산.
        optimizer.step()  # 계산된 gradient는 각 파라미터의 .grad 속성에 저장되며, 이후 optimizer.step()에서 이 값을 이용해 가중치가 갱신됨.

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)  # train 데이터에 대한 예상 라벨링
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total  # epoch 에 대한 train loss 평균
    train_acc = correct / total  # epoch 에 대한 train 정확도 측정
    train_loss.append(epoch_loss)  # train_loss 기록
    train_accuracy.append(train_acc)  # train_accuracy 기록

    model.eval()  # 검증 모드: Dropout, BatchNorm 멈춤
    correct_test = 0  # epoch 에 대한 test 적중 횟수
    total_test = 0  # 총 test 데이터 수
    with torch.no_grad():  # gradient 자동 계산 멈춤
        for images, labels in test_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)  # test 데이터에 대한 예상 라벨링
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    test_acc = correct_test / total_test
    test_accuracy.append(test_acc)  # test_accuracy 기록

    if test_acc > max_acc:  # test 성능이 제일 높은 모델 기록
        best_model = model.state_dict().copy()
        max_acc = test_acc
        torch.save(best_model, "best_model.pt")  # 모델 파라미터 정보 저장

    print(f"[Epoch {epoch + 1}/{epochs}] Loss: {epoch_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")

print(f"Training done with best test accuracy: {max_acc * 100: .4f}%")

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.title("Train Loss")
plt.plot(train_loss, label="CrossEntropyLoss")
plt.legend()

plt.subplot(1, 2, 2)
plt.title("Accuracy")
plt.plot(train_accuracy, label='train')
plt.plot(test_accuracy, label='test')
plt.legend()
plt.show()

plt.show()

## 저장된 모델 로드

In [ ]:
saved_model = SampleModel()
saved_model.load_state_dict(torch.load("best_model.pt"))
saved_model.to(device)

In [ ]:
# 로드된 모델 성능 평가
performance_check(saved_model, test_dataset)